### SPECT/CT PPV Calculation

- Calculates PPV for localized SPECT/CT scans using updated concordance logic.
- Includes exact and side matches as true positives.

Output: PPV percentage for SPECT/CT.

In [5]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [6]:
def check_concordance(row, modality_col):
    imaging = row[modality_col]
    surgical = row['Surgical Findings']

    # Return Unknown if imaging or surgical data is missing
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Handle special cases
    if imaging == 'No Scan':
        return 'No Scan'
    if imaging == 'Non-localizing':
        return 'Non-localizing'
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    imaging_glands = [g.strip().lower() for g in imaging.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]
    imaging_set = set(imaging_glands)
    surgical_set = set(surgical_glands)

    if surgical_set.issubset(imaging_set):
        return 'Yes (Exact)'
    if imaging_set & surgical_set:
        return 'Yes (Partial)'

    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]

    if any(side in surgical_sides for side in imaging_sides):
        if len(surgical_glands) > 1:
            return 'Yes (Partial)'
        else:
            return 'Yes (Side)'

    return 'Incorrect Localization'


In [ ]:
# Define modality
modality = 'SPECT/CT'

# Calculate concordance 
preop_imaging_df['SPECT/CT Calculated Concordance'] = preop_imaging_df.apply(
    lambda row: check_concordance(row, modality),
    axis=1
)

# Filter to scans that localized (exclude Non-localizing, No Scan, Unknown)
localized_spect = preop_imaging_df[
    ~preop_imaging_df['SPECT/CT Calculated Concordance'].isin(['Non-localizing', 'No Scan', 'Unknown'])
].copy()

# Count total localized scans
total_localized_spect = localized_spect.shape[0]

# Calculate true positives (Exact + Side matches)
true_positive = (
    (localized_spect['SPECT/CT Calculated Concordance'] == 'Yes (Exact)') |
    (localized_spect['SPECT/CT Calculated Concordance'] == 'Yes (Side)')
).sum()

# Calculate false positives and ppv
false_positive = (localized_spect['SPECT/CT Calculated Concordance'] == 'Incorrect Localization').sum()
ppv = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else None

print(f"Positive Predictive Value (PPV) for localized SPECT/CT scans: {ppv:.2%}" if ppv is not None else "PPV cannot be calculated (no positive or negative cases).")

Positive Predictive Value (PPV) for localized SPECT/CT scans: 90.26%
